In [1]:
from src.main import GetTanksReq, get_tank_data
from src.pool import init

await init()
req = GetTanksReq(primo_ids=["69419"])
res = await get_tank_data(req)
res

GroupingError: column "td.ts" must appear in the GROUP BY clause or be used in an aggregate function